In [1]:
from tkinter import *
from tkinter import filedialog, messagebox
from stegano import lsb
from PIL import ImageTk, Image

# Main window setup
root = Tk()
root.title("Image Steganography")
root.geometry("900x800")
root.configure(bg="#FFB6C1")

# Variables to store paths
media_path = None

# Function to open an image
def open_image():
    global media_path
    file_path = filedialog.askopenfilename(filetypes=[("PNG Files", ".png"), ("JPEG Files", ".jpg")])
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((250, 250))
        img = ImageTk.PhotoImage(img)
        media_panel.config(image=img)
        media_panel.image = img
        media_path = file_path
        placeholder_label.pack_forget()  # Hide placeholder text
        secret_text.delete("1.0", END)  # Clear text area when opening a new image

# Function to hide message in image
def hide_message_in_image():
    global media_path
    message = secret_text.get("1.0", END).strip()
    if not message:
        messagebox.showwarning("Input Error", "Enter a message to hide!")
        return
    
    if not media_path:
        messagebox.showwarning("Input Error", "Please select an image!")
        return
    
    output_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG Files", "*.png")])
    if not output_path:
        return
    
    secret_image = lsb.hide(media_path, message)
    secret_image.save(output_path)
    messagebox.showinfo("Success", f"Message hidden in image successfully!\nSaved at: {output_path}")
    
    # Clear the input fields
    secret_text.delete("1.0", END)
    media_panel.config(image='')
    media_path = None
    placeholder_label.pack(pady=20)  # Show placeholder text

# Function to reveal message from image
def reveal_message_from_image():
    global media_path
    if not media_path:
        messagebox.showwarning("Input Error", "Please select an image!")
        return
    
    try:
        hidden_message = lsb.reveal(media_path)
        if hidden_message:
            secret_text.delete("1.0", END)  # Clear the text area before displaying the new message
            secret_text.insert("1.0", hidden_message)  # Displayhe hidden message in the text area
        else:
            secret_text.delete("1.0", END)  # Clear the text area if no message is found
            secret_text.insert("1.0", "No hidden message found in the image.")
    except:
        messagebox.showerror("Error", "Make sure the image contains a hidden message.")

# GUI Layout
# Left side for Image Preview
left_frame = Frame(root, bg="#34495E",bd=5)
left_frame.place(relx=0.05, rely=0.1, relwidth=0.4, relheight=0.6)

# White background for media preview
media_preview_frame = Frame(left_frame, bg="white")
media_preview_frame.pack(padx=10, pady=10, fill=BOTH, expand=True)

# Media panel
media_panel = Label(media_preview_frame, bg="white")
media_panel.pack(padx=10, pady=10, fill=BOTH, expand=True)

# Placeholder label
placeholder_label = Label(media_preview_frame, text="Place the image here", bg="white", fg="#BDC3C7", font=("Arial", 16))
placeholder_label.pack(pady=20)

# Right side for Text Input and Buttons
right_frame = Frame(root, bg="#34495E", bd=5)
right_frame.place(relx=0.5, rely=0.1, relwidth=0.45, relheight=0.6)

Label(right_frame, text="ENTER THE MESSAGE TO HIDE:", bg="#34495E", fg="#ECF0F1", font=("Arial", 12)).pack(pady=5)

# Create a frame for the text box and scrollbar
text_frame = Frame(right_frame, bg="#34495E")
text_frame.pack(pady=10, padx=10, fill=BOTH, expand=True)

# Create text box with scrollable feature
secret_text = Text(text_frame, height=15, width=50, font=("Arial", 12), wrap=WORD, bg="#ECF0F1", fg="#2C3E50")
secret_text.pack(side=LEFT, fill=BOTH, expand=True)

# Create scrollbar and link it to text box
scrollbar = Scrollbar(text_frame, orient=VERTICAL, command=secret_text.yview)
scrollbar.pack(side=RIGHT, fill=Y)
secret_text.config(yscrollcommand=scrollbar.set)

# Bottom Buttons for File Selection and Actions
bottom_frame = Frame(root, bg="#2C3E50")
bottom_frame.place(relx=0.0, rely=0.75, relwidth=1, relheight=0.2)

# First row of buttons
btn_open_image = Button(bottom_frame, text="Open Image", command=open_image, bg="#1ABC9C", fg="white", font=("Arial", 14, "bold"))
btn_open_image.grid(row=0, column=0, padx=10, pady=10, sticky=W)

# Second row of buttons
btn_hide_image = Button(bottom_frame, text="Hide Message", command=hide_message_in_image, bg="#3498DB", fg="white", font=("Arial", 14, "bold"))
btn_hide_image.grid(row=0, column=1, padx=10, pady=10, sticky=W)

btn_reveal_image = Button(bottom_frame, text="Reveal Message", command=reveal_message_from_image, bg="#E74C3C", fg="white", font=("Arial", 14, "bold"))
btn_reveal_image.grid(row=0, column=2, padx=10, pady=10, sticky=W)

root.mainloop()
